In [4]:
import numpy as np
import time
import os
import h5py
import glob
import IPython.display
import matplotlib.pyplot as plt
np.random.seed(1337)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Merge
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.utils import np_utils
from keras import backend as K
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from PIL import Image
from keras.layers.normalization import BatchNormalization

In [5]:
# Read test images from local host
X1 = np.array(Image.open("/home/qw2208/research/left1.png"))
X1 = (X1-np.mean(X1))/np.std(X1)
X2 = np.array(Image.open("/home/qw2208/research/right1.png"))
X2 = (X2-np.mean(X2))/np.std(X2)
# input image dimensions
img_rows, img_cols = X1.shape[0], X1.shape[1]
input_shape = (1, img_rows, img_cols)

X1 = X1.reshape(1, 1, img_rows, img_cols)
X2 = X2.reshape(1, 1, img_rows, img_cols)

# number of conv filters to use
nb_filters = 112

# CNN kernel size
kernel_size = (3,3)

X1 = X1.astype('float32')
X2 = X2.astype('float32')

In [6]:
# Define CNN
left_branch = Sequential()
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same', input_shape=input_shape))
#left_branch.add(BatchNormalization(axis=1))
left_branch.add(ELU()) 
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
#left_branch.add(BatchNormalization(axis=1))
left_branch.add(ELU())
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
#left_branch.add(BatchNormalization(axis=1))
left_branch.add(ELU())
left_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
#left_branch.add(BatchNormalization(axis=1))
left_branch.add(ELU())

right_branch = Sequential()
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same', input_shape=input_shape))
#right_branch.add(BatchNormalization(axis=1))
right_branch.add(ELU())
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
#right_branch.add(BatchNormalization(axis=1))
right_branch.add(ELU())
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
#right_branch.add(BatchNormalization(axis=1))
right_branch.add(ELU())
right_branch.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same'))
#right_branch.add(BatchNormalization(axis=1))
right_branch.add(ELU())

merged = Merge([left_branch, right_branch], mode='concat', concat_axis=1)

cnn = Sequential()
cnn.add(merged)

In [7]:
def load_cnn_weights(model, filepath):
    f = h5py.File(filepath, mode='r')
    # g = f['model_weights']
    # print f["conv2d_1/conv2d_1"]
    weights = []
    for i in range(1, 9):
        weights.append(f['model_weights/convolution2d_{}/convolution2d_{}_W/'.format(i, i)][()])
        weights.append(f['model_weights/convolution2d_{}/convolution2d_{}_b/'.format(i, i)][()])
#         weights.append(f['model_weights/batchnormalization_{}/batchnormalization_{}_beta/'.format(i-2, i-2)][()])
#         weights.append(f['model_weights/batchnormalization_{}/batchnormalization_{}_gamma/'.format(i-2, i-2)][()])
#         weights.append(f['model_weights/batchnormalization_{}/batchnormalization_{}_running_mean/'.format(i-2, i-2)][()])
#         weights.append(f['model_weights/batchnormalization_{}/batchnormalization_{}_running_std/'.format(i-2, i-2)][()])
    # print len(weights)
    model.set_weights(weights)
    f.close()
    
# load weight for first cnn part
load_cnn_weights(cnn, "/home/qw2208/research/weightsOne.hdf5")

In [8]:
# predict feature map output and later will do d times fc
output_cnn = cnn.predict([X1, X2])
print "output shape is =====================>", output_cnn.shape
print (output_cnn)

output shape is =====================> (1, 224, 200, 200)
[[[[  4.10183096e+00   3.23067999e+00   3.46919656e-01 ...,
     -5.71922302e-01  -8.53164494e-01   6.03627026e-01]
   [  4.59534502e+00   2.79307652e+00   6.75977692e-02 ...,
     -9.45581436e-01  -8.54977369e-01   2.44410229e+00]
   [  1.55422139e+00  -2.40034729e-01  -3.67573172e-01 ...,
     -9.60745811e-01   6.75999746e-02   3.86205602e+00]
   ..., 
   [ -2.86317050e-01  -2.25031413e-02  -1.29979432e-01 ...,
     -1.13732470e-02   6.68197274e-01  -3.08236033e-02]
   [  2.41868898e-01   3.91618043e-01  -2.13590533e-01 ...,
      3.40103030e-01   1.93924189e-01  -5.03830791e-01]
   [  4.80399221e-01   2.38218233e-01  -1.81353435e-01 ...,
      2.74797767e-01  -6.58000186e-02  -4.23479676e-01]]

  [[  1.77062297e+00   3.28216553e+00   2.91123271e+00 ...,
     -6.34198606e-01  -7.77176797e-01  -4.60728884e-01]
   [ -9.81670916e-01  -9.99620378e-01  -9.99061108e-01 ...,
     -9.63922501e-01  -7.92149007e-01   7.87430167e-01]
   

In [11]:
# set network params for fc
nb_filters_fc = 384
kernel_size = (9, 9) 
input_shape = (nb_filters*2, None, None)

def load_fc_weights(filepath):
    f = h5py.File(filepath, mode='r')
    weights = []
    for i in range(9, 13):
        weights.append(f['model_weights/convolution2d_{}/convolution2d_{}_W/'.format(i, i)][()])
        weights.append(f['model_weights/convolution2d_{}/convolution2d_{}_b/'.format(i, i)][()])
#         weights.append(f['model_weights/batchnormalization_{}/batchnormalization_{}_beta/'.format(i-2, i-2)][()])
#         weights.append(f['model_weights/batchnormalization_{}/batchnormalization_{}_gamma/'.format(i-2, i-2)][()])
#         weights.append(f['model_weights/batchnormalization_{}/batchnormalization_{}_running_mean/'.format(i-2, i-2)][()])
#         weights.append(f['model_weights/batchnormalization_{}/batchnormalization_{}_running_std/'.format(i-2, i-2)][()])
    return weights

weights_fc = load_fc_weights("/home/qw2208/research/weightsOne.hdf5")


In [12]:
# create original fully-connected layers for training but now fully-conv layers

fc = Sequential()
fc.add(Convolution2D(nb_filters_fc, kernel_size[0], kernel_size[1], border_mode='valid', input_shape=input_shape))
# fc.add(BatchNormalization(axis=1))
fc.add(ELU())
fc.add(Convolution2D(nb_filters_fc, 1, 1, border_mode='valid'))
# fc.add(BatchNormalization(axis=1))
fc.add(ELU())
fc.add(Convolution2D(nb_filters_fc, 1, 1, border_mode='valid'))
# fc.add(BatchNormalization(axis=1))
fc.add(ELU())
fc.add(Convolution2D(1, 1, 1, border_mode='valid', activation='sigmoid'))

fc.set_weights(weights_fc)

In [14]:
# input feature map into fully-conv(test phase) layer for d times
d_max = 200
vol = np.zeros((img_rows-8, img_cols-8, d_max), dtype=np.float)
for d in range(1, d_max+1):
    input_fc_left = output_cnn[:, 0:112, :, d:]
    input_fc_right = output_cnn[:, 112:, :, 0:-d]
    input_fc = np.concatenate((input_fc_left, input_fc_right), axis=1)
    
    print input_fc.shape
    output = fc.predict(input_fc)
    print output
    vol[:, d:, d-1] = output.squeeze()
#    print "============================= ", d

(1, 224, 200, 199)
[[[[  8.55243543e-10   5.07564710e-06   5.19191497e-04 ...,
      9.76833701e-01   9.99945998e-01   9.92268026e-01]
   [  4.14851367e-01   4.93885018e-04   6.00015670e-02 ...,
      7.18966782e-01   2.99965173e-01   6.06580498e-03]
   [  2.00547929e-05   8.74323523e-05   1.91715790e-05 ...,
      8.44600677e-01   5.97788632e-01   5.57735384e-01]
   ..., 
   [  1.84467789e-02   5.26224130e-06   2.65078643e-06 ...,
      8.45627710e-07   1.33997958e-06   6.56947847e-08]
   [  9.45774264e-06   1.44349750e-08   3.92423402e-07 ...,
      1.44735637e-07   6.74598866e-09   1.23184067e-11]
   [  4.64929517e-09   3.46285788e-06   4.40857766e-05 ...,
      1.11785670e-09   1.85687455e-11   8.66547028e-14]]]]
(1, 224, 200, 198)
[[[[  1.55489834e-05   6.69910442e-05   6.29142821e-01 ...,
      9.99939203e-01   1.00000000e+00   9.98184860e-01]
   [  1.16924018e-01   4.00089994e-02   2.61302628e-02 ...,
      9.99517441e-01   9.99623895e-01   1.33146569e-01]
   [  1.01931731e-03  

ValueError: could not broadcast input array from shape (192) into shape (192,1)

In [ ]:
result_index = np.argmax(vol, axis=2)
print result_index.shape, "\n ", result_index
result_index = result_index.astype('int16')
im = Image.fromarray(result_index)
im.convert('RGB').save("disp.png")
print "Finished!"